In [97]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

from pptx import Presentation

from chat_caller import query_gpt_chat


from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

In [96]:
import os

def list_files_in_directory(directory, exclude_files=None, exclude_extensions=None):
    if exclude_files is None:
        exclude_files = ['.DS_Store']
    if exclude_extensions is None:
        exclude_extensions = ['.html', '.docx', 'odt']
    
    file_paths = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            if filename in exclude_files or any(filename.endswith(ext) for ext in exclude_extensions):
                continue
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths

In [98]:
def read_text_file(file_path):
    encodings = ['utf-8', 'latin-1', 'iso-8859-1']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        except (UnicodeDecodeError, FileNotFoundError):
            continue
    return None

In [99]:
import fitz  # PyMuPDF

def read_pdf_content(file_path):
    try:
        # Open the PDF file
        pdf_document = fitz.open(file_path)
        
        # Extract text from each page
        pdf_text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            pdf_text += page.get_text()

        return pdf_text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Replace 'your_pdf_file_path' with the actual path to your PDF file.
pdf_file_path = 'course_material/module 2/lecture slides.pdf'
pdf_content = read_pdf_content(pdf_file_path)

if pdf_content:
    print(pdf_content)

Primer on Business Analytics 
with Python
Module 2: Basic Statistical Tests and 
Regression Analysis
Introduction
- Statistical tests and regression analysis are vital tools in data science.
- They are key in hypothesis testing, understanding relationships, and making prediction
- This module will cover basic statistical tests like t-test, ANOVA, and introduce regressio
Learning Objectives
- Understand the basics of hypothesis testing.
- Learn how to perform a t-test and ANOVA in Python.
- Get introduced to linear regression and understand its applications.
Hypothesis Testing
- Hypothesis testing is a method for testing a claim or hypothesis about a parameter in a
- Null hypothesis (H0) vs Alternative hypothesis (Ha).
- Examples in business context: A/B testing, customer satisfaction analysis.
t-test
- A t-test compares the means of two groups.
- Used to determine if the means of two sets of data are significantly different from each
- Business example: Comparing the average sales befo

In [49]:


def read_pptx_content(file_path):
    try:
        # Load the PowerPoint file
        presentation = Presentation(file_path)
        
        # Extract text from each slide
        ppt_text = ""
        for slide in presentation.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    ppt_text += shape.text + "\n"

        return ppt_text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Replace 'your_pptx_file_path' with the actual path to your PowerPoint file.
pptx_file_path = 'course_material/module 3/lecture slides.pptx'
pptx_content = read_pptx_content(pptx_file_path)

if pptx_content:
    print(pptx_content)

Primer on Business Analytics with Python
Module 3: Causal Inference with Experiments and Observational Data
Introduction

- Causal inference is a fundamental concept in both statistics and business analytics.- This module will cover causal inference techniques with experiments and observational data.- Business applications include A/B testing, policy evaluation, and more.
Learning Objectives

- Understand the basics of causal inference.- Learn how to use experiments and observational data for causal inference.
Causal Inference

- Causal inference aims to identify cause-and-effect relationships between variables.- Business example: Evaluating the impact of a new marketing strategy.
Experiments and Observational Data

- Experiments involve controlled settings to determine causal effects.- Observational data comes from natural settings without controlled experiments.- Business example: A/B testing vs. retrospective studies.
Summary

- We covered the basics of causal inference, focus

In [100]:
def read_file_content(file_path):
    _, ext = os.path.splitext(file_path)
    if ext.lower() in ['.pdf']:
        return read_pdf_content(file_path)
    elif ext.lower() in ['.pptx']:
        return read_pptx_content(file_path)
    else:
        return read_text_file(file_path)

In [101]:

def create_langchain_documents(directory, exclude_files=None):
    file_paths = list_files_in_directory(directory, exclude_files)
    documents = []
    
    for file_path in file_paths:
        content = read_file_content(file_path)
        if content:
            document = LangchainDocument(page_content=content, metadata={'source': file_path})
            documents.append(document)
    
    return documents

In [102]:
directory_path = 'course_material'
exclude_files = ['.DS_Store']  # Add any other files you want to exclude
langchain_docs = create_langchain_documents(directory_path, exclude_files)

In [103]:
list_files_in_directory('course_material')

['course_material/module 5/lecture slides.pptx',
 'course_material/module 2/transcript.txt',
 'course_material/module 2/lecture slides.pdf',
 'course_material/module 3/lecture slides.pptx',
 'course_material/module 4/transcript.txt',
 'course_material/module 4/lecture slides.pptx',
 'course_material/general/syllabus.md',
 'course_material/general/available2.txt',
 'course_material/module 1/transcript.txt',
 'course_material/module 1/lecture slides.pptx',
 'course_material/module 6/transcript.txt',
 'course_material/module 6/lecture slides.pptx']

In [104]:
langchain_docs

[Document(page_content="Primer on Business Analytics with Python\nModule 5: Basic Text Analytics\nIntroduction\n\n- Text analytics deals with deriving meaningful information from textual data.\x0b- In business, text analytics can be used for customer sentiment analysis, keyword extraction, and more.\nLearning Objectives\n\n- Learn basic techniques for text analytics.\x0b- Understand how to apply text analytics in a business context.\nBasic Techniques in Text Analytics\n\n- Text Preprocessing: Tokenization, stemming, etc.\x0b- Text Vectorization: Count vectorization, TF-IDF.\x0b- Sentiment Analysis: Determining the sentiment expressed in text.\nApplications in Business\n\n- Customer Feedback Analysis: Understanding customer sentiments.\x0b- Keyword Extraction: Identifying important topics in customer reviews.\x0b- Chatbot Training: Training intelligent chatbots to handle customer queries.\nSummary\n\n- We covered basic techniques in text analytics and their applications in business.\x0b

In [105]:


langchain_docs = create_langchain_documents(directory_path, exclude_files)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

In [107]:
def call_llm(inference_client, prompt):
    response = query_gpt_chat(prompt, [])
    return response[1]

In [109]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [110]:
llm_client = None

In [32]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 15.9 MB/s eta 0:00:0000:0100:01
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install python-pptx

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [111]:
import random

N_GENERATIONS = 10  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        print("question:", question)
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 10 QA couples...


  0%|          | 0/10 [00:00<?, ?it/s]

Prompt tokens: 823
Completion tokens: 47
Total tokens: 870
question: What is the purpose of the AI chatbot in the business analytics course?

Prompt tokens: 1027
Completion tokens: 48
Total tokens: 1075
question: What are the basic techniques in text analytics covered in Module 5 of the Business Analytics with Python course?

Prompt tokens: 1035
Completion tokens: 30
Total tokens: 1065
question: What is the business example of using linear regression in the context of business analytics?

Prompt tokens: 890
Completion tokens: 43
Total tokens: 933
question: What are the three tasks in the online exam for the Business Analytics with Python course?

Prompt tokens: 1049
Completion tokens: 49
Total tokens: 1098
question: What are some common algorithms used in supervised machine learning?

Prompt tokens: 1135
Completion tokens: 56
Total tokens: 1191
question: What is the purpose of a t-test in business analytics?

Prompt tokens: 1018
Completion tokens: 54
Total tokens: 1072
question: What a

In [112]:
df_questions = pd.DataFrame(outputs)

In [113]:
df_questions

,context,question,answer,source_doc
0,- **AI Chatbot**: An AI chatbot is available f...,What is the purpose of the AI chatbot in the b...,The AI chatbot is designed to answer frequentl...,course_material/general/syllabus.md
1,Primer on Business Analytics with Python\nModu...,What are the basic techniques in text analytic...,The basic techniques in text analytics covered...,course_material/module 5/lecture slides.pptx
2,Primer on Business Analytics \nwith Python\nMo...,What is the business example of using linear r...,Predicting sales based on advertising spend.,course_material/module 2/lecture slides.pdf
3,Primer on Business Analytics with Python\nModu...,What are the three tasks in the online exam fo...,The three tasks in the online exam are Code In...,course_material/module 6/lecture slides.pptx
4,Primer on Business Analytics with Python\nModu...,What are some common algorithms used in superv...,Some common algorithms used in supervised mach...,course_material/module 4/lecture slides.pptx
5,"[Slide 1: Title Slide]\nHello again, everyone!...",What is the purpose of a t-test in business an...,A t-test allows us to compare the means of two...,course_material/module 2/transcript.txt
6,# Primer on Business Analytics with Python\n##...,What are the main concepts covered in the Busi...,The main concepts covered in the course are ba...,course_material/module 6/transcript.txt
7,"Thank you for your participation, and I'm exci...",What is the speaker excited to continue in the...,The speaker is excited to continue the learnin...,course_material/module 2/transcript.txt
8,"[Slide 1: Title Slide]\nHello, everyone! Welco...",What is the go-to tool for data manipulation a...,The go-to tool for data manipulation and analy...,course_material/module 1/transcript.txt
9,- Split data into training and test sets for u...,What is a confusion matrix in model evaluation?\n,A confusion matrix is a table that describes t...,course_material/module 4/lecture slides.pptx


In [5]:
df_questions['ques_id'] = 'Q' + (df_questions.index + 1).astype(str)

# Reorder the columns to place `item_id` after the index
columns = ['ques_id'] + df_questions.columns[:-1].tolist()  # Create the new column order list
df_questions = df_questions[columns] 

In [6]:
df_questions

,ques_id,context,question,answer,source_doc
0,Q1,- **AI Chatbot**: An AI chatbot is available f...,What is the purpose of the AI chatbot in the b...,The AI chatbot is designed to answer frequentl...,course_material/general/syllabus.md
1,Q2,Primer on Business Analytics with Python\nModu...,What are the basic techniques in text analytic...,The basic techniques in text analytics covered...,course_material/module 5/lecture slides.pptx
2,Q3,Primer on Business Analytics \nwith Python\nMo...,What is the business example of using linear r...,Predicting sales based on advertising spend.,course_material/module 2/lecture slides.pdf
3,Q4,Primer on Business Analytics with Python\nModu...,What are the three tasks in the online exam fo...,The three tasks in the online exam are Code In...,course_material/module 6/lecture slides.pptx
4,Q5,Primer on Business Analytics with Python\nModu...,What are some common algorithms used in superv...,Some common algorithms used in supervised mach...,course_material/module 4/lecture slides.pptx
5,Q6,"[Slide 1: Title Slide]\nHello again, everyone!...",What is the purpose of a t-test in business an...,A t-test allows us to compare the means of two...,course_material/module 2/transcript.txt
6,Q7,# Primer on Business Analytics with Python\n##...,What are the main concepts covered in the Busi...,The main concepts covered in the course are ba...,course_material/module 6/transcript.txt
7,Q8,"Thank you for your participation, and I'm exci...",What is the speaker excited to continue in the...,The speaker is excited to continue the learnin...,course_material/module 2/transcript.txt
8,Q9,"[Slide 1: Title Slide]\nHello, everyone! Welco...",What is the go-to tool for data manipulation a...,The go-to tool for data manipulation and analy...,course_material/module 1/transcript.txt
9,Q10,- Split data into training and test sets for u...,What is a confusion matrix in model evaluation?\n,A confusion matrix is a table that describes t...,course_material/module 4/lecture slides.pptx


In [7]:
df_questions.to_csv('questions.csv', index=False)